In [18]:

import os

import keras
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import layers
from keras import models

os.environ["TF_USE_LEGACY_KERAS"] = "1"

## Load dataset

In [14]:
(ds_train, ds_validation, ds_test), metadata  = tfds.load(
    "tf_flowers", 
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], 
    as_supervised=True, 
    with_info=True
)

In [3]:
metadata

In [15]:
print(metadata.splits["train"].num_examples)
num_classes = metadata.features["label"].num_classes
labels = metadata.features["label"].names
print(num_classes)
print(labels)

## Visualize a few images

In [16]:

samples = ds_train.take(4)
fig = plt.figure(figsize=(5, 5))
for i,img in enumerate(tfds.as_numpy(samples)):
    img_array, img_label_idx = img
    ax = fig.add_subplot(2,2, i + 1)    
    ax.imshow(img_array)    
    ax.set_title(labels[img_label_idx])
plt.show()

## Prepare dataset for training

In [24]:
IMG_SIZE = 180

resize_and_rescale = models.Sequential([
    layers.Resizing(IMG_SIZE,IMG_SIZE),
    layers.Rescaling(1./255)
])

data_augmentation = models.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1)
])

def prepare_for_training(ds_subset, batch_size = 32, shuffle=False, augment=False):
    ds_subset  =  ds_subset.map(lambda x, y: (resize_and_rescale(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # shuffle the dataset if needed
    if shuffle:
        ds_subset = ds_subset.shuffle(1000)

    # create data batches
    ds_subset = ds_subset.batch(batch_size)

    # apply data augmentation
    if augment:        
        ds_subset= ds_subset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds_subset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


transformed_ds_train = prepare_for_training(ds_train, shuffle=True, augment=True)
transformed_ds_val = prepare_for_training(ds_validation)
transformed_ds_test = prepare_for_training(ds_test)

### Check how the images look like after processing

In [25]:
samples = transformed_ds_train.take(4)
fig = plt.figure(figsize=(5, 5))
for i,img in enumerate(tfds.as_numpy(samples)):
    batch_images, batch_labels = img
    #print(np.min(img_array), np.max(img_array))
    ax = fig.add_subplot(2,2, i + 1)    
    ax.imshow(batch_images[0])    
    ax.set_title(labels[batch_labels[0]])
plt.show()

### Create model

In [27]:
model = models.Sequential([    
  layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input"),  
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [28]:

model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
model.summary()

### Train model and visualize training history

In [29]:
epochs=20
history = model.fit(transformed_ds_train,validation_data=transformed_ds_val, epochs=epochs)

In [30]:
history_dict = history.history
val_acc_values = history_dict['val_accuracy']
train_acc_values = history_dict['accuracy']
val_loss_values = history_dict['val_loss']
train_loss_values = history_dict['loss']
epochs_range = range(epochs)


In [31]:
## Plotting the training and validation loss

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_loss_values, 'r', label='Training loss')
plt.plot(epochs_range, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

## Plotting the training and validation accuracy

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_acc_values, 'r', label='Training accuracy')
plt.plot(epochs_range, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Generic Metrics

In [32]:
## create confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

y = [labels for imgs, labels in tfds.as_numpy(transformed_ds_test.unbatch())]
y_pred_probs = model.predict(transformed_ds_test)
y_pred = np.argmax(y_pred_probs, axis=1)


fig, _ = plt.subplots(nrows=1, figsize=(10,10))
ax = plt.subplot(1, 1, 1)
ax.grid(False)
cf = confusion_matrix(y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cf, display_labels=labels)
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=ax)
plt.show()

### Inspect model parameters

Inspect the model parameters and see how many parameters are there in the model.


In [33]:
model.summary()

### Inspect learned features

In [34]:

image_url = "https://static.wixstatic.com/media/e6591e_3a5449fe774a4993a1166b1d33e233f7~mv2_d_1880_1253_s_2.jpg/v1/fill/w_1000,h_666,al_c,q_90,usm_0.66_1.00_0.01/e6591e_3a5449fe774a4993a1166b1d33e233f7~mv2_d_1880_1253_s_2.jpg"
image_path = tf.keras.utils.get_file("image_test", image_url)
img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))

In [35]:
plt.imshow(img)

In [36]:
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_tensor = tf.convert_to_tensor(img_array)
img_tensor = tf.expand_dims(img_tensor, 0) 
img_tensor = tf.divide(img_tensor, 255.0)
scores = model(img_tensor)
probs = tf.nn.softmax(scores).numpy().squeeze()
class_idx = np.argmax(probs)
print(f"this image was classified as {labels[class_idx]} with a probability of {probs[class_idx]}")

In [37]:
import math
cnn_layers = [layer for layer in model.layers  if isinstance(layer, layers.Conv2D)] 
selected_layers = cnn_layers[:1]       
features_extraction_model = models.Sequential(selected_layers)
features_map = features_extraction_model(img_tensor).numpy().squeeze()
num_fmaps = features_map.shape[-1]
num_rows = math.ceil(num_fmaps / 6)
fig = plt.figure(figsize=(10,10))
for i, fmap in enumerate(np.rollaxis(features_map, axis=2)):
    ax = fig.add_subplot(num_rows, 6, i + 1)
    ax.grid(False)
    ax.imshow(fmap, cmap='viridis')
plt.show()

In [38]:
cnn_layer  = model.get_layer("conv2d_3")
features_map = cnn_layer(img_tensor)
features_map = features_map.numpy().squeeze()
fig = plt.figure(figsize=(10,10))
for i, fmap in enumerate(np.rollaxis(features_map, axis=2)):
    ax = fig.add_subplot(4,4, i + 1)
    ax.grid(False)
    ax.imshow(fmap, cmap='viridis')
plt.show()

### Save model

In [44]:
from pathlib import Path
MODELS_DIR = Path("models/flowers-model")
MODELS_DIR.mkdir(exist_ok=True, parents=True)
TF_MODEL_PATH = MODELS_DIR.joinpath("model.keras")

In [45]:
model.save(TF_MODEL_PATH)